## 1.Jupyter

In [ ]:
# [use this command once for ipython-sql installation] 
# !conda install -yc conda-forge ipython-sql

In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re

## 2.Introduction

We'll use the following code to connect our Jupyter Notebook to our database file:

In [1]:
%%capture
%load_ext sql
%sql sqlite:///factbook.db

## 3.Overview of the Data

* Write a query to return information on the tables in the database.
* In a different code cell, write and run another query that returns the first five rows of the facts table in the database.

In [2]:
%%sql

SELECT *
  FROM sqlite_master
 WHERE type='table';

 * sqlite:///factbook.db
Done.


type,name,tbl_name,rootpage,sql
table,sqlite_sequence,sqlite_sequence,3,"CREATE TABLE sqlite_sequence(name,seq)"
table,facts,facts,47,"CREATE TABLE ""facts"" (""id"" INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL, ""code"" varchar(255) NOT NULL, ""name"" varchar(255) NOT NULL, ""area"" integer, ""area_land"" integer, ""area_water"" integer, ""population"" integer, ""population_growth"" float, ""birth_rate"" float, ""death_rate"" float, ""migration_rate"" float)"


In [11]:
%%sql
SELECT *
FROM facts
LIMIT 5;

 * sqlite:///factbook.db
Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
1,af,Afghanistan,652230,652230,0,32564342,2.32,38.57,13.89,1.51
2,al,Albania,28748,27398,1350,3029278,0.3,12.92,6.58,3.3
3,ag,Algeria,2381741,2381741,0,39542166,1.84,23.67,4.31,0.92
4,an,Andorra,468,468,0,85580,0.12,8.13,6.96,0.0
5,ao,Angola,1246700,1246700,0,19625353,2.78,38.78,11.49,0.46


In [8]:
facts_df = pd.read_sql_table(table_name='facts', con='sqlite:///factbook.db')
facts_df

,id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
0,1,af,Afghanistan,652230.0,652230.0,0.0,3.256434e+07,2.32,38.57,13.89,1.51
1,2,al,Albania,28748.0,27398.0,1350.0,3.029278e+06,0.30,12.92,6.58,3.30
2,3,ag,Algeria,2381741.0,2381741.0,0.0,3.954217e+07,1.84,23.67,4.31,0.92
3,4,an,Andorra,468.0,468.0,0.0,8.558000e+04,0.12,8.13,6.96,0.00
4,5,ao,Angola,1246700.0,1246700.0,0.0,1.962535e+07,2.78,38.78,11.49,0.46
...,...,...,...,...,...,...,...,...,...,...,...
256,257,zh,Atlantic Ocean,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
257,258,xo,Indian Ocean,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
258,259,zn,Pacific Ocean,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
259,260,oo,Southern Ocean,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Here are the descriptions for some of the columns:

* name — the name of the country.
* area — the total land and sea area of the country.
* population — the country's population.
* population_growth— the country's population growth as a percentage.
* birth_rate — the country's birth rate, or the number of births a year per 1,000 people.
* death_rate — the country's death rate, or the number of death a year per 1,000 people.
* area— the country's total area (both land and water).
* area_land — the country's land area in square kilometers.
* area_water — the country's water area in square kilometers.

Let's start by calculating some summary statistics and see what they tell us.

## 4.Summary Statistics

Write a single query that returns the following:

* Minimum population
* Maximum population
* Minimum population growth
* Maximum population growth

### using pandas

In [16]:
facts_df.dtypes

id                     int64
code                  object
name                  object
area                 float64
area_land            float64
area_water           float64
population           float64
population_growth    float64
birth_rate           float64
death_rate           float64
migration_rate       float64
dtype: object

In [18]:
facts_df[facts_df["population"]==facts_df["population"].min()]

,id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
249,250,ay,Antarctica,NaN,280000.0,NaN,0.0,NaN,NaN,NaN,NaN


In [19]:
facts_df[facts_df["population"]==facts_df["population"].max()]

,id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
260,261,xx,World,NaN,NaN,NaN,7.256490e+09,1.08,18.6,7.8,NaN


In [21]:
facts_df[facts_df["population_growth"]==facts_df["population_growth"].min()]

,id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
189,190,vt,Holy See (Vatican City),0.0,0.0,0.0,842.0,0.0,NaN,NaN,NaN
199,200,ck,Cocos (Keeling) Islands,14.0,14.0,0.0,596.0,0.0,NaN,NaN,NaN
206,207,gl,Greenland,2166086.0,2166086.0,NaN,57733.0,0.0,14.48,8.49,5.98
237,238,pc,Pitcairn Islands,47.0,47.0,0.0,48.0,0.0,NaN,NaN,NaN


In [20]:
facts_df[facts_df["population_growth"]==facts_df["population_growth"].max()]

,id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
161,162,od,South Sudan,644329.0,NaN,NaN,12042910.0,4.02,36.91,8.18,11.47


### using sql

In [27]:
%%sql

SELECT MIN(population) AS min_pop,
       MAX(population) AS max_pop,
       MIN(population_growth) AS min_pop_growth,
    MAX(population_growth) AS max_pop_growth 
FROM facts;

 * sqlite:///factbook.db
Done.


min_pop,max_pop,min_pop_growth,max_pop_growth
0,7256490011,0.0,4.02


A few things are interesting in the summary statistics on the previous screen:

* There's a country with a population of 0.
* There's a country with a population of 7256490011 (or more than 7.2 billion people).
Let's use subqueries to concentrate on these countries without using the specific values.

## 5.Exploring Outliers

* Write a query that returns the countries with the minimum population.
* Write a query that returns the countries with the maximum population.

### using sql

In [29]:
%%sql

select *
from facts
where population = (select MIN(population)
                       FROM facts);



 * sqlite:///factbook.db
Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
250,ay,Antarctica,None,280000,None,0,None,None,None,None


In [30]:
%%sql

select *
from facts
where population = (select max(population)
                       FROM facts);



 * sqlite:///factbook.db
Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
261,xx,World,None,None,None,7256490011,1.08,18.6,7.8,None


Recompute the summary statistics you found earlier while excluding the row for the whole world. Include the following:

* Minimum population
* Maximum population
* Minimum population growth
* Maximum population growth

### using pandas

In [50]:
display(facts_df[facts_df["population"]==facts_df["population"].min()])
display(facts_df[facts_df["name"].str.lower().str.contains("world")==False].nlargest(1,"population")) 
display(facts_df[facts_df["population_growth"]==facts_df["population_growth"].min()])
display(facts_df[facts_df["population_growth"]==facts_df["population_growth"].max()])

,id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
249,250,ay,Antarctica,NaN,280000.0,NaN,0.0,NaN,NaN,NaN,NaN


,id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
36,37,ch,China,9596960.0,9326410.0,270550.0,1.367485e+09,0.45,12.49,7.53,0.44


,id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
189,190,vt,Holy See (Vatican City),0.0,0.0,0.0,842.0,0.0,NaN,NaN,NaN
199,200,ck,Cocos (Keeling) Islands,14.0,14.0,0.0,596.0,0.0,NaN,NaN,NaN
206,207,gl,Greenland,2166086.0,2166086.0,NaN,57733.0,0.0,14.48,8.49,5.98
237,238,pc,Pitcairn Islands,47.0,47.0,0.0,48.0,0.0,NaN,NaN,NaN


,id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
161,162,od,South Sudan,644329.0,NaN,NaN,12042910.0,4.02,36.91,8.18,11.47


### using sql

In [61]:
%%sql

SELECT MIN(population) AS min_pop,
       MAX(population) AS max_pop,
       MIN(population_growth) AS min_pop_growth,
    MAX(population_growth) AS max_pop_growth 

FROM facts
WHERE NOT name LIKE "World" ;

 * sqlite:///factbook.db
Done.


min_pop,max_pop,min_pop_growth,max_pop_growth
0,1367485388,0.0,4.02


## 6.Exploring Average Population and Area

calculate the average value for the following columns:

* population
* area

### using pandas

In [66]:
display(  facts_df["population"].mean() )
display(  facts_df["area"].mean() )

62094928.32231405

555093.546184739

### using sql

In [72]:
%%sql

SELECT AVG(population) AS avg_population,
       AVG(area) AS avg_area

FROM facts
WHERE NOT name LIKE "World" ;

 * sqlite:///factbook.db
Done.


avg_population,avg_area
32242666.56846473,555093.546184739


## 7.Finding Densely Populated Countries

Write a query that finds all countries meeting both of the following criteria:
* The population is above average.
* The area is below average.

In [76]:
%%sql

SELECT *

FROM facts
WHERE NOT name LIKE "World" AND
      population > 32242666.56846473 AND
      area < 555093.546184739;

 * sqlite:///factbook.db
Done.


id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
14,bg,Bangladesh,148460,130170,18290,168957745,1.6,21.14,5.61,0.46
65,gm,Germany,357022,348672,8350,80854408,0.17,8.47,11.42,1.24
80,iz,Iraq,438317,437367,950,37056169,2.93,31.45,3.77,1.62
83,it,Italy,301340,294140,7200,61855120,0.27,8.74,10.19,4.1
85,ja,Japan,377915,364485,13430,126919659,0.16,7.93,9.51,0.0
91,ks,"Korea, South",99720,96920,2800,49115196,0.14,8.19,6.75,0.0
120,mo,Morocco,446550,446300,250,33322699,1.0,18.2,4.81,3.36
138,rp,Philippines,300000,298170,1830,100998376,1.61,24.27,6.11,2.09
139,pl,Poland,312685,304255,8430,38562189,0.09,9.74,10.19,0.46
163,sp,Spain,505370,498980,6390,48146134,0.89,9.64,9.04,8.31


Some of these countries are generally known to be densely populated, so we have confidence in our results!

## 8.next steps

1. Which country has the most people? Which country has the highest growth rate?
2. Which countries have the highest ratios of water to land? Which countries have more water than land?
3. Which countries will add the most people to their populations next year?
4. Which countries have a higher death rate than birth rate?
5. Which countries have the highest population/area ratio, and how does it compare to list we found in the previous screen?